<a href="https://colab.research.google.com/github/simonemallei/complex-systems-social-graph/blob/main/recommender_social_graph/multi_dimensional/notebook/multi_opinions.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [39]:
nodes, opinions = 5, 2
g = create_graph(nodes, ops=opinions)
simulate_epoch_updated(g, opinions, 50, 60)
polarisation(g)

POST  {3: [[-0.06, -0.88]], 4: [[-0.06, -0.88]]}
opinion [[0.18, -0.94], [-0.06, -0.88], [-0.64, -0.4], [-0.68, -0.82], [-0.28, 0.36]]


array([[0.54832],
       [1.18352]])

In [2]:
#@title
import networkx as nx
import random
from collections import defaultdict
import matplotlib.pyplot as plt
import numpy as np
from tabulate import tabulate
import math

In [3]:
#@title
def MY_homophilic_ba_graph(N, m, ops=1, alpha=2, beta=1):
    """Return homophilic random graph using BA preferential attachment model.
    A graph of n nodes is grown by attaching new nodes each with m
    edges that are preferentially attached to existing nodes with high
    degree. The connections are established by linking probability which 
    depends on the connectivity of sites and the homophily(similarities).
    homophily varies ranges from 0 to 1.

    Parameters
    ----------
    N : int
        Number of nodes
    m : int
        Number of edges to attach from a new node to existing nodes
    ops : int
        Number of opinions of each node
   """

    G = nx.Graph()
    node_attribute = {}
    

    for n in range(N):
        #generate opinion 
        op = [int(random.random()*100) for i in range(ops)]
        G.add_node(n , opinion = op)
        node_attribute[n] = op

    #create homophilic distance ### faster to do it outside loop ###
    dist = defaultdict(int) #distance between nodes

    #euclidean distance
    for n1 in range(N):
        n1_attr = node_attribute[n1]
        for n2 in range(N):
            n2_attr = node_attribute[n2]
            op_distance = 0
            for o in range(ops):
              op_distance += (n1_attr[o] - n2_attr[o]) ** 2
            dist[(n1,n2)] = math.sqrt(op_distance)

    target_list = list(range(m))
    source = m #start with m nodes

    while source < N:
        targets = _pick_targets(G,source,target_list,dist,m, alpha=alpha ,beta=beta)
        if targets != set(): #if the node does  find the neighbor
            G.add_edges_from(zip([source]*m,targets))

        target_list.append(source)  #tagrte list is updated with all the nodes in the graph 
        source += 1
    return G




In [4]:
#@title
def _pick_targets(G,source,target_list,dist,m ,alpha, beta):
    '''
    First compute the target_prob which is related to the degree'''
    target_prob_dict = {}
    for target in target_list:
        pow_dist =  (dist[(source,target)]+1)**alpha
        target_prob = (1/pow_dist)*((G.degree(target)+0.00001)**beta) #formula to compute targer prob, >>Degree better chance
        target_prob_dict[target] = target_prob
        
    prob_sum = sum(target_prob_dict.values())

    targets = set()
    target_list_copy = target_list.copy()
    count_looking = 0
    if prob_sum == 0:
        return targets #it returns an empty set

    while len(targets) < m:
        count_looking += 1
        if count_looking > len(G): # if node fails to find target
            break
        rand_num = random.random()
        cumsum = 0.0
        for k in target_list_copy:
            cumsum += float(target_prob_dict[k]) / prob_sum
            if rand_num < cumsum:  ### ??????????
                targets.add(k)
                target_list_copy.remove(k)
                break
    return targets

In [5]:
#@title
def create_graph(n_ag, ops=1,  beba_beta=[1] , avg_friend=3, hp_alpha=2, hp_beta=1):
  # checks on beba_beta length
  if len(beba_beta) != 1 and len(beba_beta) != n_ag:
    print("WARNING: beba_beta length is not valid. It must be 1 or nodes' number. Default value will be used")
    beba_beta = [1]
    beba_beta = [beba_beta[0] for node in range(n_ag)]

  if len(beba_beta) == 1:
    beba_beta = [beba_beta[0] for node in range(n_ag)]

  # Calls MY_homophilic_ba_graph
  G = MY_homophilic_ba_graph(n_ag, avg_friend, ops, hp_alpha, hp_beta)
  
  # Remapping opinions from [0, 100] to [-1, 1]
  users_opinions = nx.get_node_attributes(G, 'opinion')
  for user in users_opinions:
    for current_op in range(ops):
      users_opinions[user][current_op] = (users_opinions[user][current_op] - 50) / 50

  # Setting opinions as node attributes  
  nx.set_node_attributes(G, users_opinions, 'opinion')

  # Setting beba_beta as node attributes
  node_beba_beta_dict = dict(zip(G.nodes(), beba_beta))
  nx.set_node_attributes(G, node_beba_beta_dict, 'beba_beta')



  return G

In [6]:
#@title
def compute_activation(G, nodes, ops):
  opinions = nx.get_node_attributes(G, 'opinion')
  all_feeds = nx.get_node_attributes(G, 'feed')
  beba_beta_list = nx.get_node_attributes(G, 'beba_beta')
  # Activating update of each node
  for curr_node in nodes:
    node_feeds = all_feeds.get(curr_node, [])

    # Computing weight w(i, i)
    weight_noose = beba_beta_list[curr_node] * np.dot(opinions[curr_node], opinions[curr_node]) + 1

    # Computing new opinion of curr_node
    op_num = [weight_noose * op for op in opinions[curr_node]]
    op_den = weight_noose
    for feed in node_feeds:
      # Computing weights w(i, j) where i == curr_node and y(j) == feed
      weight = beba_beta_list[curr_node] * np.dot(feed, opinions[curr_node]) + 1
      for i in range(ops):
        op_num[i] += weight * feed[i]
      op_den += weight

    # If the denominator is < 0, the opinion gets polarized and 
    # the value is set to sgn(opinions[curr_node])
    if op_den <= 0:
      for current_op in range(ops):
        opinions[curr_node][current_op] = opinions[curr_node][current_op] / abs(opinions[curr_node][current_op])
    else:
      for current_op in range(ops):
        opinions[curr_node][current_op] = op_num[current_op] / op_den
  
    # Opinions are capped within [-1, 1] 
    for current_op in range(ops):
      if opinions[curr_node][current_op] < -1:
        opinions[curr_node][current_op] = -1
      if opinions[curr_node][current_op] > 1:
        opinions[curr_node][current_op] = 1
    all_feeds[curr_node] = []
  
  # Updating feed and opinion attributes
  nx.set_node_attributes(G, all_feeds, 'feed')
  nx.set_node_attributes(G, opinions, 'opinion')

  return G

In [7]:
#@title
def compute_post(G, nodes, ops, epsilon = 0.0):
  opinions = nx.get_node_attributes(G, 'opinion')
  for node_id in nodes:
    new_opinion = []
    for op in range(ops):
      rand_eps = np.random.normal(0, epsilon, 1)
      noise_op = rand_eps[0] + opinions[node_id][op]
      noise_op = min(noise_op, 1)
      noise_op = max(noise_op, -1)
      new_opinion.append(noise_op)

    post = [new_opinion]
    past_feed = nx.get_node_attributes(G, 'feed')

    #Spread Opinion
    all_neig = list(nx.neighbors(G, node_id))   #get all neighbours ID

    
    post_to_be_added = dict(zip(all_neig,
                                   [list(post) for _ in range(len(all_neig))] ))

    post_post_to_be_added = {key: past_feed[key] + value 
                              if key in [*past_feed]
                              else value
                              for key, value in post_to_be_added.items()}
      
    print('POST ',  post_post_to_be_added)
    nx.set_node_attributes(G, post_post_to_be_added , name='feed')
  return G

In [8]:
#@title
def simulate_epoch_updated(G, ops, percent_updating_nodes, percent_posting_nodes, epsilon = 0.0):
  # Sampling randomly the activating nodes
  updating_nodes = int(percent_updating_nodes * len(G.nodes()) / 100)
  act_nodes = np.random.choice(range(len(G.nodes())), size=updating_nodes, replace=False)

  # Executing activation phase: activated nodes will consume their feed
  G = compute_activation(G, act_nodes, ops)

  # Sampling randomly the posting nodes from activating nodes' list
  posting_nodes = int(percent_posting_nodes * len(act_nodes) / 100)
  post_nodes = np.random.choice(act_nodes,size=posting_nodes, replace = False)

  # Executing posting phase: activated nodes will post in their neighbours' feed
  G = compute_post(G, post_nodes, ops, epsilon)
  return G

In [9]:
#@title
def apply_initial_feed(G, ops, n_post = 10, epsilon = 0.1):
  # Casting all the numpy arrays as built-in lists 
  initial_feed_dict = dict()
  opinions = nx.get_node_attributes(G, 'opinion')

  for curr_node in G.nodes():
    # Sampling {n_post} elements from a normal distribution with
    # - mean = 0.0
    # - std = epsilon
    # This values are added with the original opinion in order to have 
    # a feed that has similar values with the starting opinion
    feed = [np.random.normal(0, epsilon, ops) + opinions[curr_node] for i in range(n_post)]
    for i in range(n_post):
      for j in range(ops):
        feed[i][j] = max(-1, min(1, feed[i][j]))
    initial_feed_dict[curr_node] = list(feed)
  
  # Setting these values as feed in the graph
  nx.set_node_attributes(G, initial_feed_dict, 'feed')

  return G

In [10]:
#@title
# create graph and update it with ABEBA model (with epsilon-error == 0.0)
nodes, ops = 10, 3
G = create_graph(nodes, ops, [1], avg_friend = 3, hp_alpha=5, hp_beta=0)
G = apply_initial_feed(G, ops, n_post=2)
print("Starting graph: ")
labels =  nx.get_node_attributes(G, 'opinion')
print(tabulate([[key] + [np.round(val, 3)] for key, val in labels.items()], headers=["node label", "opinion value"]))

Starting graph: 
  node label  opinion value
------------  -------------------
           0  [0.94 0.98 0.18]
           1  [ 0.9  -0.72 -0.68]
           2  [ 0.66  0.58 -0.8 ]
           3  [-0.06 -0.74 -0.2 ]
           4  [-0.06 -0.38 -0.8 ]
           5  [ 0.04 -0.02 -0.22]
           6  [-0.32  0.48  0.16]
           7  [ 0.74  0.82 -0.32]
           8  [-0.38 -0.34 -0.94]
           9  [-0.04 -0.34 -0.02]


In [22]:
#@title
# Simulating an epoch and printing the opinion graph obtained
G = simulate_epoch_updated(G, ops, 50, 50)
labels =  nx.get_node_attributes(G, 'opinion')
print(tabulate([[key] + [np.round(val, 3)] for key, val in labels.items()], headers=["node label", "opinion value"]))

POST  {4: [[-0.3678049634988393, -0.35198244122564576, -0.9279605256726843]]}
POST  {3: [[0.5009278483124533, 0.3726960154118406, -0.6644794089702246]], 4: [[-0.3678049634988393, -0.35198244122564576, -0.9279605256726843], [0.5009278483124533, 0.3726960154118406, -0.6644794089702246]], 6: [array([-0.2453081 ,  0.35885751,  0.00577946]), array([-0.40521002,  0.43459899,  0.04014651]), [0.07572503804364532, -0.01281566559334495, -0.2961225922591539], [0.5009278483124533, 0.3726960154118406, -0.6644794089702246]], 7: [[0.07572503804364532, -0.01281566559334495, -0.2961225922591539], [0.5009278483124533, 0.3726960154118406, -0.6644794089702246]]}
  node label  opinion value
------------  ----------------------
           0  [0.94 0.98 0.18]
           1  [ 0.9  -0.72 -0.68]
           2  [ 0.501  0.373 -0.664]
           3  [ 0.004 -0.451 -0.24 ]
           4  [-0.024 -0.383 -0.441]
           5  [ 0.076 -0.013 -0.296]
           6  [-0.32  0.48  0.16]
           7  [ 0.741  0.89  -0.4  ]


In [38]:
def polarisation(G):
  opinions = list(nx.get_node_attributes(G, 'opinion').values())
  print('opinion', opinions)
  ops = len(opinions[0])
  n = len(opinions)
  means = np.zeros((ops, 1))
  for i in range(n):
    for j in range(ops):
      means[j] += opinions[i][j]
  for j in range(ops):
    means[j] /= n
  pol = np.zeros((ops, 1))
  for i in range(n):
    for j in range(ops):
      pol[j] += (opinions[i][j] - means[j]) ** 2
  return pol